In [1]:
# !pip install selenium

  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/10/56/8288d1813a68c1e0638515dbb777fce6d87d0d240e683216f956145310e6/selenium-4.11.2-py3-none-any.whl.metadata
  Obtaining dependency information for urllib3[socks]<3,>=1.26 from https://files.pythonhosted.org/packages/9b/81/62fd61001fa4b9d0df6e31d47ff49cfa9de4af03adecf339c7bc30656b37/urllib3-2.0.4-py3-none-any.whl.metadata
  Obtaining dependency information for trio~=0.17 from https://files.pythonhosted.org/packages/a3/dd/b61fa61b186d3267ef3903048fbee29132963ae762fb70b08d4a3cd6f7aa/trio-0.22.2-py3-none-any.whl.metadata
  Obtaining dependency information for trio-websocket~=0.9 from https://files.pythonhosted.org/packages/a5/a6/06e2373f95c12e9e8f6b910a76c86e375348ead77ab476230640666310fb/trio_websocket-0.10.3-py3-none-any.whl.metadata
  Obtaining dependency information for certifi>=2021.10.8 from https://files.pythonhosted.org/packages/4c/dd/2234eab22353ffc7d94e8d13177aaa050113286e93e7b40eae01fbf

In [21]:
#!pip install pandas

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/b7/f8/32d6b5aa4c4bc045fa2c4c58f88c325facc54721956c6313f0afea8ea853/pandas-2.1.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for numpy>=1.23.2 from https://files.pythonhosted.org/packages/72/b2/02770e60c4e2f7e158d923ab0dea4e9f146a2dbf267fec6d8dc61d475689/numpy-1.25.2-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/502.3 kB ? eta -:--:--
     -- ------------------------------------ 30.7/502.3 kB 1.3 MB/s eta 0:00:01
     ----------------- -------------------- 225.3/502.3 kB 3.5 MB/s eta 0:00:01
     -------------------------------------- 502.3/502.3 kB 4.5 MB/s eta 0:00:00
     ---------------------------------------- 0.0/341.8 kB ? eta -:--:--
     ------------------------------------- 341.8/341.8 kB 10.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   -- ------------------------------------- 0

In [22]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep

In [2]:
driver = webdriver.Edge()

In [3]:
driver.get("https://chess.com/events/2023-bullet-chess-championship-main-event/games")

Prima di continuare, bisogna fare un scroll-down per caricare gli elementi

In [4]:
# Trova tutti gli elementi con la classe round-games-list-item-games
game_items = driver.find_elements(By.CLASS_NAME, "round-games-list-item-games")

In [5]:
# Crea una lista vuota per i link
links = []

# Per ogni elemento game_item, trova tutti gli elementi a con l'attributo href
for game_item in game_items:
    game_links = game_item.find_elements(by=By.TAG_NAME, value="a")

    # Aggiungi tutti i link alla lista
    for game_link in game_links:
        links.append(game_link.get_attribute("href"))

if len(links) > 31:
    links = links[:32]
    print('Success')
else:
    print('Load more matches')

Success


## Extract informations

In [17]:
# Variables for summary Dataframe
player = []
color_generic = []
clock_value = []
clock_result = []
accuracy_score = []
game = []

# Variables for in-game specifics Dataframe
move = []
skill = []
turn = []
color = []
match = []
n = 0


for link in links:
    # Load the page
    driver.get(link)
    sleep(5)
    

    # Save the player ...
    if 'Nakamura_Hikaru-Carlsen_Magnus' in links[0]:
        player.append('Hikaru Nakamura')
        player.append('Magnus Carlsen')
    elif 'Carlsen_Magnus-Nakamura_Hikaru' in links[0]:
        player.append('Magnus Carlsen')
        player.append('Hikaru Nakamura')
    # ... and the piece colors
    color_generic.append('W')
    color_generic.append('B')

    # Saving clock-values and results
    for i in range(1, -1, -1):
        clock_value.append(driver.find_elements(By.CLASS_NAME, 'clock-value')[i].text)
        clock_result.append(driver.find_elements(By.CLASS_NAME, 'clock-result')[i].text)
    
    # Saving the accuracy
    for i in range(2):
        accuracy_score.append(driver.find_elements(By.CLASS_NAME, 'accuracy-score-value')[i].text)
    
    # Then save the game
    n += 1
    game.append(n)
    game.append(n)
    

    # Load the match-analysis tab
    driver.find_element(By.CSS_SELECTOR, "button[data-cy='game-sidebar-tab-analysis']").click()
    sleep(3)

    # Takes moves in the game analysis
    moves_driver = driver.find_elements(By.CLASS_NAME, 'move-node')
    moves = []
    for m in moves_driver:
        moves.append(m.find_element(By.CLASS_NAME, 'move-text').text)
        # Check if the move is 'bad/good' or 'normal'
        if 'customColor' in m.get_attribute('class'):
            # Get style
            style = m.get_attribute('style')
            index = style.find('#')
            skill.append(style[index:index + 7])
        else:
            skill.append('=')
    
    # Storing all moves
    move += moves

    # Saving the colors and the number of the game
    for i in range(len(moves)):
        turn.append(int(i/2) + 1)
        if i % 2 == 0:
            color.append('W')
        else:
            color.append('B')
        match.append(n)
    
    print('Game', n, 'fatto')

Game 1 fatto
Game 2 fatto
Game 3 fatto
Game 4 fatto
Game 5 fatto
Game 6 fatto
Game 7 fatto
Game 8 fatto
Game 9 fatto
Game 10 fatto
Game 11 fatto
Game 12 fatto
Game 13 fatto
Game 14 fatto
Game 15 fatto
Game 16 fatto
Game 17 fatto
Game 18 fatto
Game 19 fatto
Game 20 fatto
Game 21 fatto
Game 22 fatto
Game 23 fatto
Game 24 fatto
Game 25 fatto
Game 26 fatto
Game 27 fatto
Game 28 fatto
Game 29 fatto
Game 30 fatto
Game 31 fatto
Game 32 fatto


In [19]:
print(len(player))
print(len(color_generic))
print(len(clock_value))
print(len(clock_result))
print(len(accuracy_score))
print(len(game))

print(len(move))
print(len(skill))
print(len(turn))
print(len(color))
print(len(match))

64
64
64
64
64
64
3439
3439
3439
3439
3439


In [24]:
df_summary = pd.DataFrame({
    'player': player,
    'color_generic': color_generic,
    'clock_value': clock_value,
    'clock_result': clock_result,
    'accuracy_score': accuracy_score,
    'game': game
})

df_summary

,player,color_generic,clock_value,clock_result,accuracy_score,game
0,Hikaru Nakamura,W,0:30,DRAW,96.1,1
1,Magnus Carlsen,B,0:25,DRAW,96.0,1
2,Hikaru Nakamura,W,0:05.1,DRAW,80.4,2
3,Magnus Carlsen,B,0:05.1,DRAW,78.2,2
4,Hikaru Nakamura,W,0:37,WON,92.2,3
...,...,...,...,...,...,...
59,Magnus Carlsen,B,0:17.5,LOST,82.7,30
60,Hikaru Nakamura,W,0:03.5,WON,79.8,31
61,Magnus Carlsen,B,0:00,LOST,79.3,31
62,Hikaru Nakamura,W,0:09.8,LOST,92.4,32


In [27]:
df_games = pd.DataFrame({
    'move': move,
    'skill': skill,
    'turn': turn,
    'color': color,
    'match': match
})

df_games

,move,skill,turn,color,match
0,e4,=,1,W,1
1,e5,=,1,B,1
2,Nf3,=,2,W,1
3,Nc6,=,2,B,1
4,d4,=,3,W,1
...,...,...,...,...,...
3434,Qd2+,=,56,B,32
3435,Kf2,=,57,W,32
3436,c2,=,57,B,32
3437,Qc7,=,58,W,32


In [30]:
df_games.skill = df_games.skill.str.strip()

In [32]:
df_summary.to_csv('summary.csv', index=False)
df_games.to_csv('games.csv', index=False)

In [33]:
df_games.skill.value_counts()

skill
=          3058
#e6912c     141
#5c8bb0     130
#ee6b55      81
#b33430      25
#1baca6       4
Name: count, dtype: int64

In [35]:
df_games[df_games.skill == '#1baca6']

,move,skill,turn,color,match
1166,Bxh7+,#1baca6,23,W,11
1607,Nxg6,#1baca6,31,W,16
2173,Nxd5,#1baca6,18,B,21
3406,Qxe4,#1baca6,42,B,32


In [36]:
symbol = ['??', '?', 'X', '!', '!!']
meaning = ['Blunder', 'Mistake', 'Missed win', 'Good move', 'Brilliant move']
hex = ['#b33430', '#e6912c', '#ee6b55', '#5c8bb0', '#1baca6']

df_annotation = pd.DataFrame({
    'symbol': symbol,
    'meaning': meaning,
    'hex': hex
})

df_annotation

,symbol,meaning,hex
0,??,Blunder,#b33430
1,?,Mistake,#e6912c
2,X,Missed win,#ee6b55
3,!,Good move,#5c8bb0
4,!!,Brilliant move,#1baca6


In [37]:
df_annotation.to_csv('annotation.csv')